Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.113788
 0.287772
 0.562692

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 2.01555  1.6418    0.65297 
 1.6418   1.36588   0.555696
 0.65297  0.555696  0.232594

In [6]:
det(Σ[2])

0.008343359741668693

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...);

In [8]:
rr = RandomNode(aa, Dict{Factor, Symbol}());

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...);

In [12]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.11378754588892104,0.28777209609605703,0.5626916910279065]   …  [0.15941594910798207,0.32110166213092395,0.06689663508541632]
 [0.14599995288817613,0.9339680927598752,0.06642259413619533]      [0.2741934065327303,0.2365511184198652,0.902461531680802]    
 [0.05411992760880602,0.2853934392111066,0.024992104181771557]     [0.7405211631317117,0.24366582622599542,0.11714905049712798] 
 [0.09435500639551186,0.5458053427442064,0.7300453408813254]       [0.035629931347286314,0.615538227580865,0.772040419697849]   
 [0.6106189798052677,0.41508750464901034,0.6710434853941385]       [0.2956610718721058,0.053608398593532725,0.8807027418773983] 

In [13]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}

# Make a factor and check its value

In [14]:
μ = rand(10)
τ = rand(10)
x = RandomNode(Normal, rand(10), rand(10))
f = LogNormalFactor(x, μ, τ)

VB.LogNormalFactor(VB.RandomNode{Distributions.Normal,1}([Distributions.Normal(μ=0.6627835774427986, σ=0.28093909224835145),Distributions.Normal(μ=0.35451098626614574, σ=0.2948281680578657),Distributions.Normal(μ=0.1078871659744205, σ=0.698740080207491),Distributions.Normal(μ=0.14307895877371668, σ=0.30306984120619473),Distributions.Normal(μ=0.5432347344764896, σ=0.964323801550002),Distributions.Normal(μ=0.39623520979132, σ=0.4425538805629492),Distributions.Normal(μ=0.6194873263202185, σ=0.911385480560382),Distributions.Normal(μ=0.3982997351734372, σ=0.08206025953432339),Distributions.Normal(μ=0.4533498383708128, σ=0.40979817985566447),Distributions.Normal(μ=0.13464535817930168, σ=0.03391301962971127)],Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,1}([0.7421418282158763,0.8382179684475022,0.1975888571540132,0.15860904219649985,0.44376195014627706,0.8874845794606436,0.3324763481173525,0.5488048511563073,0.7060844201379459,0.1586517157483005],Dict{VB.Factor,Symbol}()),VB.ConstantNode

In [15]:
value(f)

-4.975481871254818

In [16]:
register(f)
x.factormap

Dict{VB.Factor,Symbol} with 1 entry:
  VB.LogNormalFactor(VB.R… => :x

In [17]:
g = EntropyFactor(x)
value(g)

2.4254781911019156

In [18]:
naturals(x)

10-element Array{Tuple{Float64,Float64},1}:
 (8.39744927613165,-6.334985930498852)    
 (4.078418147974703,-5.752174553079815)   
 (0.22097262469342693,-1.0240913397698224)
 (1.557723343566101,-5.443579394611347)   
 (0.5841733742412296,-0.5376804327545368) 
 (2.023115876317646,-2.5529228931764214)  
 (0.7458100218253015,-0.6019574494408507) 
 (59.14856943656302,-74.2513291037052)    
 (2.699562676024019,-2.9773504339665604)  
 (117.07346261494669,-434.74748850623126) 

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [19]:
dims = (5, 6)

# note: it won't matter much how we initialize here
μ = RandomNode(Normal, zeros(dims), ones(dims))
τ = RandomNode(Gamma, ones(dims), ones(dims))

y = rand(dims)

5x6 Array{Float64,2}:
 0.0786174  0.215739  0.618774  0.696846  0.752616  0.340406
 0.473923   0.701501  0.669439  0.615966  0.997821  0.927259
 0.659904   0.751534  0.152288  0.801711  0.864923  0.247305
 0.910839   0.264615  0.535305  0.632064  0.849096  0.063461
 0.878099   0.863109  0.754667  0.457505  0.894104  0.81031 

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [20]:
obs = LogNormalFactor(y, μ, τ)
μ_prior = LogNormalFactor(μ, zeros(dims), 2 * ones(dims))
τ_prior = LogGammaFactor(τ, 1.1 * ones(dims), ones(dims))

VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,Dict{VB.Factor,Symbol}()))

In [21]:
value(obs), value(μ_prior), value(τ_prior)

(-40.6800529706196,-67.96536370453937,7.496173237795199)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.R… => :x
  VB.LogNormalFactor(VB.C… => :μ

In [24]:
naturals(obs, Val{:x}, μ)

(
5x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [25]:
naturals(obs, Val{:μ}, μ)

(
5x6 Array{Float64,2}:
 0.0786174  0.215739  0.618774  0.696846  0.752616  0.340406
 0.473923   0.701501  0.669439  0.615966  0.997821  0.927259
 0.659904   0.751534  0.152288  0.801711  0.864923  0.247305
 0.910839   0.264615  0.535305  0.632064  0.849096  0.063461
 0.878099   0.863109  0.754667  0.457505  0.894104  0.81031 ,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [26]:
naturals(obs, Val{:τ}, τ)

(0.5,
5x6 Array{Float64,2}:
 0.50309   0.523272  0.69144   0.742797  0.783215  0.557938
 0.612302  0.746052  0.724074  0.689707  0.997824  0.929905
 0.717737  0.782402  0.511596  0.821371  0.874046  0.53058 
 0.914814  0.535011  0.643276  0.699752  0.860482  0.502014
 0.885529  0.872478  0.784761  0.604655  0.899711  0.828301)